# Basic Python Programming 004 Custom Dataset

In [4]:
import os, glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from PIL import Image
from numpy import asarray

#### custom dataset for image classification
Chest X-RAY-images (Pneumonia classification) (dataset from https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia)

1. class Dataset: Use dataset class to store data with preprocessing (options: normalize, shuffle)
2. class Dataloader: load dataset in numpy array with batch_size

In [5]:
os.listdir('chest_xray/train')

['PNEUMONIA', 'NORMAL']

In [6]:
labels = os.listdir('chest_xray/train/')
for label in labels:
    print(label)
    globals()[label]=0

print(NORMAL)

PNEUMONIA
NORMAL
0


In [7]:
img = Image.open("chest_xray/train/NORMAL/IM-0280-0001.jpeg")
img = img.resize((1200,900))
img.save('temp.jpg')
img = asarray(img)

In [8]:
img.shape

(900, 1200)

In [46]:
from typing import Any, Callable, Iterable, TypeVar, Generic, Sequence, List, Optional, Union

class DataLoader():
    def __init__(self, PATH,
                 batch_size:Optional[int]=1,
                 normalize:Optional[bool]=False,
                 shuffle:Optional[bool]=False,
                 get_statistics:Optional[bool]=False # to get mean and standard deviation of datas
                 ):

        # get NORMAL and PNEUMONIA folder path
        self.NORMAL_PATH = os.path.join(PATH, "NORMAL")
        self.PNEUMONIA_PATH = os.path.join(PATH, "PNEUMONIA")
        # get paths for normal images and pneumonia images
        self.NORMAL_PATHS = glob.glob(os.path.join(self.NORMAL_PATH, "*.jpeg"))
        self.PNEUMONIA_PATHS = glob.glob(os.path.join(self.PNEUMONIA_PATH, "*.jpeg"))
        # concatenate images path and make label datas 
        self.images_path = self.NORMAL_PATHS + self.PNEUMONIA_PATHS
        self.labels = [0] * len(self.NORMAL_PATHS) + [1] * len(self.PNEUMONIA_PATHS)
        
        # zip images_path and labels for shuffling
        self.datas = list(zip(self.images_path, self.labels))

        self.normalize_flag = normalize
            
        if shuffle:
            self.datas = shuffle(self.datas)

        if get_statistics:
            self.mu = 0.0
            self.sd = 0.0

            for i, (image_path, label) in enumerate(self.datas):
                image = Image.open(image_path)
                image = image.resize((1200,900))
                image = np.asarray(image)
                self.mu += np.sum(image)

            self.mu /= ((i+1)*1200*90)
            print(self.mu)

    def normalize(self, img_path):
        pass

    def load(self, batch_size):
        pass


In [47]:
train_loader = DataLoader(PATH="chest_xray/train", normalize=True)

1341.1327180768071


In [21]:
print(len(train_loader.NORMAL_PATHS))
print(len(train_loader.PNEUMONIA_PATHS))
print(len(train_loader.labels))

1341
2101
3442


In [35]:
train_loader

NameError: name 'image' is not defined